# Import Libraries & Dataset

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

2022-08-01 13:47:57.159664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:47:57.159683: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Dataset
df = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# Split the dataset
X = df[:, 0:8]
Y = df[:,-1]

# Fix the random seed for reproducability
seed = 7
tf.random.set_seed(seed)

In [3]:
df.shape

(768, 9)

## Keras Models in scikit-learn

Keras models must be wrapped in either **KerasClassifier** or **KerasRegressor** class from the **SciKeras** module. To utilize these wrappers we need to define a function that creates and returns the Keras sequential model, then pass this function to the `model` argument when construction the **KerasClassifier** class.

In [4]:
# function to create a model as mentioned above
def create_model():
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Grid Search

The constructor for the KerasClassifier class can take new arguments that can be passed to your custom `create_model()` function. These new args must also be defined in the signature of your `create_model()` function with default parameters

## Tuning Batch Size and # of Epochs

In [7]:
model = KerasClassifier(model=create_model, verbose=0)

# Define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

2022-07-31 17:24:58.218537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:24:58.218537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:24:58.218559: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-31 17:24:58.218561: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-31 17:24:58.218565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or dire

2022-07-31 17:25:00.107579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 17:25:00.107988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:25:00.108043: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-31 17:25:00.108092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-31 17:25:00.108135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [8]:
# Summarizing the results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"{round(mean, 6)} {round(stdev, 6)} with {param}")

Best: 0.6888020833333334 using {'batch_size': 10, 'epochs': 100}
0.615885 0.009207 with {'batch_size': 10, 'epochs': 10}
0.670573 0.02436 with {'batch_size': 10, 'epochs': 50}
0.688802 0.046475 with {'batch_size': 10, 'epochs': 100}
0.561198 0.040133 with {'batch_size': 20, 'epochs': 10}
0.654948 0.047771 with {'batch_size': 20, 'epochs': 50}
0.658854 0.060037 with {'batch_size': 20, 'epochs': 100}
0.561198 0.071342 with {'batch_size': 40, 'epochs': 10}
0.648438 0.008438 with {'batch_size': 40, 'epochs': 50}
0.68099 0.010253 with {'batch_size': 40, 'epochs': 100}
0.588542 0.028587 with {'batch_size': 60, 'epochs': 10}
0.626302 0.065468 with {'batch_size': 60, 'epochs': 50}
0.626302 0.025582 with {'batch_size': 60, 'epochs': 100}
0.488281 0.11163 with {'batch_size': 80, 'epochs': 10}
0.571615 0.067231 with {'batch_size': 80, 'epochs': 50}
0.667969 0.027251 with {'batch_size': 80, 'epochs': 100}
0.578125 0.051329 with {'batch_size': 100, 'epochs': 10}
0.59375 0.013902 with {'batch_size':

## Tuning Learning Rate and Momentum

Using the above results we will now test for the optimal learning rate and momentum for the SGD optimizer.  Momentum controls how much to let the previous update influence the current weight update. For this step we will be assuming that Stochastic Gradient Descent is the most optimal optimizer to use which in most cases will be correct. 

In [9]:
# Make a new create_model() function as this one won't include the .compile() call 
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In the above `create_model()` we don't include the .compile() unlike earlier as its better to leave the optimizer for a Keras model to the KerasClassifier

In [10]:
model = KerasClassifier(model=create_model, 
                        loss="binary_crossentropy", 
                        optimizer="SGD", 
                        epochs=100, 
                        batch_size=10, 
                        verbose=0)

lr = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
# With the SciKeras wrapper, we will route the parameters 
# to the optimizer with the prefix optimizer__
param_grid = dict(optimizer__learning_rate=lr, optimizer__momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

2022-07-31 17:30:43.360499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:30:43.360494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:30:43.360494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:30:43.360494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:30:43.360519: I tensorflow/stream_executor/cuda/cudart_stub.cc:29]

2022-07-31 17:30:45.250339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-31 17:30:45.250693: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-31 17:30:45.250732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-31 17:30:45.250767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-31 17:30:45.250797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [11]:
# Summarize the Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.678385 using {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.643229 (0.031948) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.635417 (0.024360) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.665365 (0.034987) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.654948 (0.030145) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.678385 (0.020505) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.641927 (0.068207) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.652344 (0.011500) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.667969 (0.005524) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.658854 (0.018136) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.4}
0.649740 (0.003683) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.6}
0.651042 (0.001841)

## Tuning Network Weight Initialization

Weight initialization refers to the process of randomly initializing the network's 
weights and biases. There are many methods to doing this ie. HeNormal, GlorotUniform, RandomUniform etc. 

In [4]:
# Define a new create_model() function to accept the initialization technique as parameter
def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [5]:
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# Defining the GridSearch parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero','glorot_normal',
            'glorot_uniform', 'he_normal','he_uniform']
param_grid = dict(model__init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,cv=3)
grid_result = grid.fit(X, Y)

2022-08-01 13:52:44.323203: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:52:44.323207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:52:44.323202: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:52:44.323241: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-01 13:52:44.323241: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your mac

2022-08-01 13:52:46.363022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 13:52:46.364159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:52:46.364355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-01 13:52:46.364510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-01 13:52:46.364670: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

2022-08-01 13:52:57.096981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 13:52:57.097245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 13:52:57.097274: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-01 13:52:57.097298: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-01 13:52:57.097319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [6]:
# Summarizing Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.716146 using {'model__init_mode': 'normal'}
0.707031 (0.024910) with: {'model__init_mode': 'uniform'}
0.670573 (0.047771) with: {'model__init_mode': 'lecun_uniform'}
0.716146 (0.014382) with: {'model__init_mode': 'normal'}
0.651042 (0.001841) with: {'model__init_mode': 'zero'}
0.709635 (0.006639) with: {'model__init_mode': 'glorot_normal'}
0.700521 (0.031948) with: {'model__init_mode': 'glorot_uniform'}
0.678385 (0.018136) with: {'model__init_mode': 'he_normal'}
0.665365 (0.035849) with: {'model__init_mode': 'he_uniform'}


## Tuning the Neuron Activation Function

The activation function controls the non-linearity of individual neurons and when to fire. Normally the `rectifier` or `sigmoid` are most popular. The GridSearch will only be testing the activation function for the hidden layers as the output layer needs to have an activation function used for binary classification. 

Note that generally each activation function will perform differently due to range of data that the neuron will be inputted. Some activation functions need the input to be standardized and others don't

In [12]:
def create_model(activation='relu'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer='uniform', activation=activation))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [13]:
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(model__activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [14]:
# Summarize Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.721354 using {'model__activation': 'softplus'}
0.653646 (0.001841) with: {'model__activation': 'softmax'}
0.721354 (0.019488) with: {'model__activation': 'softplus'}
0.680990 (0.013279) with: {'model__activation': 'softsign'}
0.703125 (0.029232) with: {'model__activation': 'relu'}
0.675781 (0.008438) with: {'model__activation': 'tanh'}
0.699219 (0.030425) with: {'model__activation': 'sigmoid'}
0.671875 (0.032369) with: {'model__activation': 'hard_sigmoid'}
0.705729 (0.001841) with: {'model__activation': 'linear'}


## Tuning Dropout Regularization


Dropout is a regularization technique where randomly selected neurons are ignored during training. They are "dropped-out" randomly. This means that their contribution to the activation of downstream neurons is temporarily removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.

As the neural network learns, neuron weights settle into their context  within the network. Weights of neurons are tuned for specific features providing some specialization. Neighboring neurons become to rely on this specialization, which if taken too far can result in a fragile model too specialized to the training data. This reliance on context for a specific neuron during training is referred to complex *co-adaptations*.

If neurons are randomly dropped out of the network during training, than other neurons will have to step in and handle the representation required to make predictions for the missing neurons. By doing this no neuron will be too independent of its neighboring neurons. This is believed to result in multiple independent internal reperesentation being learned by the network.

The effect is that the network becomes less sensitive to the specific weights of neurons. This in turn results in a network that is capable of better generalization and is less likely to overfit the training data.
[Credit](https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/)

In [16]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

def create_model(dropout_rate, weight_constraint):
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [17]:
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(model__dropout_rate=dropout_rate, model__weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

2022-08-01 14:43:48.511895: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:43:48.512180: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-01 14:43:48.527355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:43:48.527357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:43:48.527355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror:

2022-08-01 14:43:50.451554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.451721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.451778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.451790: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup th

2022-08-01 14:43:50.724046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 14:43:50.725097: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.725589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.725934: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-01 14:43:50.726190: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [18]:
# Summarize the Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.731771 using {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.731771 (0.012890) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 1.0}
0.713542 (0.008027) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 2.0}
0.707031 (0.011500) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 3.0}
0.713542 (0.010253) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 4.0}
0.712240 (0.004872) with: {'model__dropout_rate': 0.0, 'model__weight_constraint': 5.0}
0.704427 (0.003683) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 1.0}
0.717448 (0.012075) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 2.0}
0.709635 (0.004872) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 3.0}
0.704427 (0.009744) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 4.0}
0.710938 (0.017758) with: {'model__dropout_rate': 0.1, 'model__weight_constraint': 5.0}
0.709635 (0.023073) with: {'model__dr

## Tuning the Number of Neurons in the Hidden Layer

The number of neuruons in the hidden layers control the representation capacity of the network, at least at that point in the topology. A large enough single layer (in theory) can approximate any other neural network. For this case we will be try values between 1 and 30 in steps of 5. The larger the network the more training will be needed; and ideally the batch size and number of epochs should be optimized with the number of neurons.

In [19]:
def create_model(neurons):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(8,), kernel_initializer='uniform', activation='linear', kernel_constraint=MaxNorm(4)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(model__neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

2022-08-01 14:50:09.061382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:50:09.061382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:50:09.061381: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:50:09.061381: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:50:09.061407: I tensorflow/stream_executor/cuda/cudart_stub.cc:29]

2022-08-01 14:50:10.978659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-01 14:50:10.989447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 14:50:10.990150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 14:50:10.990395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such 

In [21]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.718750 using {'model__neurons': 10}
0.691406 (0.011500) with: {'model__neurons': 1}
0.705729 (0.001841) with: {'model__neurons': 5}
0.718750 (0.014616) with: {'model__neurons': 10}
0.707031 (0.011500) with: {'model__neurons': 15}
0.713542 (0.009744) with: {'model__neurons': 20}
0.694010 (0.012890) with: {'model__neurons': 25}
0.688802 (0.023510) with: {'model__neurons': 30}


[Credit](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)